# Useless (But Fun 🙃) Feature Engineering!

The titanic dataset had already been investigated from the ground up.
Everything useful about the dataset was already known.

Everything **Useful**..

This notebook is about **useless** features. These features by no means are useful and should not be used in a predictive model.

They are just a fun joke to play with, Let's see how far can we go with this ;) 

In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

df = pd.read_csv('../input/titanic-extended/full.csv')

In [ ]:
df['Name'] = df['Name'].str.upper()
df['Surname'] = [df.iloc[i]['Name'].split(',')[0] for i in range(len(df))]
df['FirstName'] = [df.iloc[i]['Name'].split('.')[1].split('(')[0].strip() for i in range(len(df))]

combined = df #  train_data.append(test_data)

def assign_couples(row):
    findmatch=combined[(combined.Sex=='female') & (combined.Name.str.contains('\)')) & (combined.Surname==row['Surname']) & (combined.FirstName==row['FirstName'])]
    if len(findmatch) == 1:
        coupleid='Couple_'+ row['Surname'] + "_" + str(findmatch.iloc[0]['PassengerId']) + "_" + str(row['PassengerId'])
        combined.loc[combined.PassengerId==findmatch.iloc[0]['PassengerId'],'Couple']=coupleid
        combined.loc[combined.PassengerId==row['PassengerId'], 'Couple'] = coupleid

combined[combined.Sex=='male'].apply(assign_couples, axis=1)
combined.loc[combined.PassengerId==1185, 'Couple'] = 'Couple_Dodge_1266_1185'.upper()
combined.loc[combined.PassengerId==1266, 'Couple'] = 'Couple_Dodge_1266_1185'.upper()
combined.loc[combined.PassengerId==446, 'Couple'] = None
combined.loc[combined.PassengerId==257, 'Couple'] = 'Couple_Thorne_257_1219'.upper()
combined.loc[combined.PassengerId==1219, 'Couple'] = 'Couple_Thorne_257_1219'.upper()
combined.loc[combined.PassengerId == 200, 'Couple'] = 'Couple_Harbeck_200_1265'.upper()
combined.loc[combined.PassengerId == 1265, 'Couple'] = 'Couple_Harbeck_200_1265'.upper()
combined.loc[combined.PassengerId == 428, 'Couple'] = 'Couple_Marshall_428_706'.upper()
combined.loc[combined.PassengerId == 706, 'Couple'] = 'Couple_Marshall_428_706'.upper()
combined.loc[combined.PassengerId==539, 'Couple'] = 'Couple_Risien_1274_539'.upper()
combined.loc[combined.PassengerId==1274, 'Couple'] = 'Couple_Risien_1274_539'.upper()

# Has a cousin on board?
### A passenger has a cousin passenger on board 

**First,** let's find parents for each passenger. BUT not the usual way since we are trying to find parents AND grandparents this time. 
We will do so by trying to estimate if there is another person on board that has the same surname and as the passenger's father or mother and then check this person's kids.

> Currently, there are no cousin on board.


In [ ]:
def check_if_adult_parent(row):
    try:
        find_child = combined[(combined.Name.str.contains('\(')) & (combined.Surname==row['Surname']) & (combined.FirstName==row['FirstName'])]
        find_adult_parent = combined[(combined.Age>=30) & (combined.Name.str.contains('\)')) & (combined.Surname==find_child.iloc[0]['Surname']) & (combined.FirstName==find_child.iloc[0]['FirstName'])]
        if row['Age'] >= find_adult_parent.iloc[0]['Age']:
            return True
        else:
            return False
    except:
        return False

In [ ]:
combined['has_adult_parent'] = combined.apply(check_if_adult_parent, axis=1)

def check_if_any_children(row):
    try:
        find_child = combined[(combined.Name.str.contains('\(')) & (combined.Couple==row['Couple'])]
        find_adult_parent = combined[(combined.Age>=30) & (combined.Name.str.contains('\)')) & (combined.Surname==find_child.iloc[0]['Surname']) & (combined.FirstName==find_child.iloc[0]['FirstName'])]
        find_brothers_sisters = combined[(combined.Surname==find_adult_parent.iloc[0]['Surname']) & (combined.FirstName!=find_adult_parent.iloc[0]['FirstName']) & (combined.Age>=30)]
        find_children = combined[(combined.Name.str.contains('\(')) & (combined.Surname==find_brothers_sisters.iloc[0]['Surname']) & (combined.FirstName==find_brothers_sisters.iloc[0]['FirstName'])]
        if len(find_children) >= 1:
            return True
        else:
            return False
    except:
        return False

In [ ]:
combined['has_children'] = combined.apply(check_if_any_children, axis=1)
combined['has_cousin'] = combined['has_adult_parent'] & combined['has_children']
train = df

### Is Rich
##### Travelling first class AND Alone OR with a nanny AND spent more than 40 dollars

In [ ]:
def is_rich(train):
    # Rich: travelling in first class AND alone or with a nanny AND spent more than 40 dollars
    train['is_rich'] = np.where((train['Pclass'] == 1) &
                                (train['SibSp'] < 1) &
                                (train['Parch'] < 1) &
                                (train['Fare'] > 40),
                                1, 0)
    return train['is_rich']

train['is_rich'] = is_rich(train)

### Stole the ticket
##### Spent less than 5 dollars

In [ ]:
def stole_the_ticket(train):
    # Stole the ticket: Spent less than 5 dollars
    train['stole_the_ticket'] = np.where((train['Fare'] < 5),
                                         1, 0)
    return train['stole_the_ticket']

train['stole_the_ticket'] = stole_the_ticket(train)

### Daddy's ATM
##### First class AND teenager AND alone or with a nanny AND spent more than 40 dollars

In [ ]:
def daddys_atm(train):
    # Daddy's ATM: First class AND teenager AND alone or with a nanny AND spent more than 40 dollars
    train['daddys_atm'] = np.where((train['Age'] < 16) &
                                   (train['SibSp'] < 1) &
                                   (train['Parch'] < 1) &                                   
                                   (train['Fare'] > 40),
                                   1, 0)
    return train['daddys_atm']

train['daddys_atm'] = daddys_atm(train)

### Business as usual
##### Men paid less than 10 dollars who also weren't travelling with their family.

In [ ]:
def business_as_usual(train):
    # Business as usual: Men with less than 10 dollars who also weren't travelling with their family.
    train['business_as_usual'] = np.where((train['Sex'] == 'male') &
                                          (train['SibSp'] + train ['Parch']) < 1 &
                                          (train['Fare'] < 10),
                                          1, 0)
    return train['business_as_usual']

train['business_as_usual'] = business_as_usual(train)

### Party All Night
##### Third class AND alone AND spent less than 5 dollars

In [ ]:
def party_all_night(train):
    # Party all night: Third class AND alone AND spent less than 5 dollars
    train['party_all_night'] = np.where((train['Pclass'] == 3) &
                                        (train['SibSp'] < 1) &
                                        (train['Parch'] < 1) &
                                        (train['Fare'] < 5),
                                        1, 0)
    return train['party_all_night']

train['party_all_night'] = party_all_night(train)

### Why me
##### A person that traveled in third class alone

In [ ]:
def why_me(train):
    # Why me: A person that traveled in third class alone.
    train['why_me'] = np.where((train['Pclass'] == 3) &
                               (train['SibSp'] == 0) &
                               (train['Parch'] == 0),
                               1, 0)
    return train['why_me']

train['why_me'] = why_me(train)

### The Gang
##### Third class AND travelled with 5+ family members AND spent less than 5 dollars.

In [ ]:
def the_gang(train):
    # The gang: Third class AND travelled with 5+ family members AND spent less than 5 dollars.
    train['the_gang'] = np.where((train['Pclass'] == 3) &
                                 (train['SibSp'] + train ['Parch']) > 4 &
                                 (train['Fare'] < 5),
                                 1, 0)
    return train['the_gang']

train['the_gang'] = the_gang(train)

### Let Them Think
##### A person that boarded in Southampton AND was a child AND was travelling alone.

In [ ]:
def let_them_think(train):
    # Let them think: A person that boarded in Southampton AND was a child AND was travelling alone.
    train['let_them_think'] = np.where((train['Boarded'] == 'Southampton') &
                                       (train['Age'] < 16) &
                                       (train['SibSp'] < 1) &
                                       (train['Parch'] < 1),
                                       1, 0)
    return train['let_them_think']

train['let_them_think'] = let_them_think(train)

### What A Coincidence
##### Travelling with a woman and she's not his wife.

In [ ]:
def what_a_coincidence(train):
    # What a coincidence: travelling with a woman and she's not his wife.
    train['what_a_coincidence'] = np.where((train['Sex'] == 'male') &
                                           (train['SibSp'] == 0) &
                                           (train['Parch'] == 1),
                                           1, 0)
    return train['what_a_coincidence']

train['what_a_coincidence'] = what_a_coincidence(train)

### Nanny In Every Corner
##### Travelling first class AND have at least one nanny

In [ ]:
def nanny_in_every_corner(train):
    # Nanny in every corner: travelling first class AND have at least one nanny
    train['nanny_in_every_corner'] = np.where((train['Pclass'] == 1) &
                                              (train['SibSp'] > 0) &
                                              (train['Parch'] > 0),
                                              1, 0)
    return train['nanny_in_every_corner']

train['nanny_in_every_corner'] = nanny_in_every_corner(train)

### Nothing But The Best
##### A person that paid an unusually high price (z-score) for his ticket.

In [ ]:
def nothing_but_the_best(train):
    # Nothing but the best: A person that paid an unusually high price (z-score) for his ticket.
    train['nothing_but_the_best'] = np.where((train['Fare'] > train['Fare'].mean() + 3 * train['Fare'].std()), 1, 0)
    return train['nothing_but_the_best']

train['nothing_but_the_best'] = nothing_but_the_best(train)

### It's My Party
##### A male traveling with 4 or more women.

In [ ]:
def its_my_party(train):
    train['is_male'] = np.where((train['Sex'] == 'male'), 1, 0)
    women_alone = train[(train['Sex'] == 'female') & (train['SibSp'] == 0) & (train['Parch'] == 0)]
    women_alone['is_travelling_with_someone'] = 0
    women_together = train[(train['Sex'] == 'female') & (train['SibSp'] > 0) & (train['Parch'] > 0)]
    women_together['is_travelling_with_someone'] = 1
    women_group = women_alone.append(women_together)
    women_group = women_group.groupby(['Ticket'])['is_travelling_with_someone'].sum().reset_index()
    women_group = women_group.merge(train, on='Ticket', how='inner')
    women_group = women_group.drop_duplicates()
    women_group = women_group.rename(columns={'is_travelling_with_someone': 'women_in_room'})
    men_with_women = women_group[women_group['is_male'] == 1]
    men_with_women['its_my_party'] = np.where((men_with_women['women_in_room'] > 3), 1, 0)
    train = train.merge(men_with_women[['PassengerId', 'its_my_party']], on='PassengerId', how='left')
    train['its_my_party'] = train['its_my_party'].fillna(0)
    return train['its_my_party']

train['its_my_party'] = its_my_party(train)

### Who the hell are you?
##### A person sharing the room with someone that is not from their family and not a nanny or child[insert]


In [ ]:
def who_the_hell_are_you(train):
    # Who the hell are you?: A person sharing the room with someone that is not from their family and not a nanny or child.
    train['who_the_hell_are_you'] = np.where((train['Pclass'] > 1) &
                                             (train['SibSp'] + train ['Parch']) > 1 &
                                             (train['Fare'] > 10),
                                             1, 0)
    return train['who_the_hell_are_you']

train['who_the_hell_are_you'] = who_the_hell_are_you(train)

### Pimp My Ride
##### Third class AND teenager AND Alone AND Spent more than 5 dollars

In [ ]:
def pimp_my_ride(train):
    # Pimp my ride: Third class AND teenager AND Alone AND Spent more than 5 dollars
    train['pimp_my_ride'] = np.where((train['Pclass'] == 3) &
                                     (train['Age'] < 16) &
                                     (train['SibSp'] < 1) &
                                     (train['Parch'] < 1) &
                                     (train['Fare'] > 5),
                                     1, 0)
    return train['pimp_my_ride']

train['pimp_my_ride'] = pimp_my_ride(train)

### Boy's Club
##### Teenager travelling with at least one other teenager AND the sum of their fares is higher than 15.

In [ ]:
def boys_club(train):
    boys_together = train[(train['Age'] < 16) & (train['SibSp'] > 0) & (train['Parch'] > 0)]
    boys_together['boys_together'] = 1
    boys_alone = train[(train['Age'] < 16) & (train['SibSp'] == 0) & (train['Parch'] == 0)]
    boys_alone['boys_together'] = 0
    boys_group = boys_together.append(boys_alone)
    boys_group = boys_group.groupby(['Ticket'])['boys_together'].sum().reset_index()
    boys_group = boys_group.merge(train, on='Ticket', how='inner')
    boys_group = boys_group.drop_duplicates()
    boys_group = boys_group.rename(columns={'boys_together': 'boys_in_room'})
    boys_with_money = boys_group[boys_group['boys_in_room'] > 1]
    boys_with_money['boys_club'] = np.where((boys_with_money['Fare'] > 15), 1, 0)
    train = train.merge(boys_with_money[['PassengerId', 'boys_club']], on='PassengerId', how='left')
    train['boys_club'] = train['boys_club'].fillna(0)
    return train['boys_club']

train['boys_club'] = boys_club(train)

### Don't wanna know
##### A person is a male and travelling with two or more women AND paid less than 5 dollars.

In [ ]:
def dont_wanna_know(train):
    # Don't wanna know: A person is a male and travelling with two or more women AND paid less than 5 dollars.
    # Let's think step by step
    # Step 1: Is a male?
    train['is_male'] = np.where((train['Sex'] == 'male'), 1, 0)
    # Step 2: Is travelling with 2 or more women?
    train['travelling_with_2_or_more_women'] = np.where((train['SibSp'] + train['Parch']) > 1, 1, 0)
    # Step 3: Paid less than 5 dollars?
    train['paid_less_than_5_dollars'] = np.where((train['Fare'] < 5), 1, 0)
    # Let's merge all the information
    train['dont_wanna_know'] = train['is_male'] & train['travelling_with_2_or_more_women'] & train['paid_less_than_5_dollars']
    return train['dont_wanna_know']

train['dont_wanna_know'] = dont_wanna_know(train)

### Wait a sec. hold on!
##### Young male alone with nannies.

In [ ]:
def wait_a_sec_hold_on(train):
    # Wait a sec. hold on!: Young male alone with nannies everywhere (or at least one nanny).
    train['wait_a_sec_hold_on'] = np.where((train['Age'] < 30) &
                                           (train['Sex'] == 'male') &
                                           (train['SibSp'] == 0) &
                                           (train['Parch'] == 1),
                                           1, 0)
    return train['wait_a_sec_hold_on']

train['wait_a_sec_hold_on'] = wait_a_sec_hold_on(train)

### Left the kids downstairs.
##### Parents that have much better room than their children.

In [ ]:
def left_the_kids_downstairs(train):
    k_num = train['SibSp'] + train['Parch']
    k_quality = train['Fare'] / k_num
    train['k_num'] = k_num
    train['kids_room_quality'] = k_quality
    train['left_the_kids_downstairs'] = np.where((train['k_num'] > 0) & (train['Pclass'] < train['kids_room_quality']), 1, 0)
    return train['left_the_kids_downstairs']

train['left_the_kids_downstairs'] = left_the_kids_downstairs(train)

### YOLO
##### A person who paid a lower fare but with a better room quality 

In [ ]:
def yolo(train):
    # YOLO: A person who paid a lower fare but with a better room quality than another passenger.
    train['yolo'] = np.where((train['Fare'] < train['Fare'].mean() - 3 * train['Fare'].std()) &
                             (train['Pclass'] > 1),
                             1, 0)
    return train['yolo']

train['yolo'] = yolo(train)

### Someone Made A Fortune On This Guy's Back
##### A person paid an unusually high price (z-score) for his ticket. 


In [ ]:
def someone_made_a_fortune_on_this_guys_back(train):
    # A person paid an unusually high price (z-score) for his ticket.
    train['someone_made_a_fortune_on_this_guys_back'] = np.where((train['Fare'] > train['Fare'].mean() + 3 * train['Fare'].std()), 1, 0)
    return train['someone_made_a_fortune_on_this_guys_back']

train['someone_made_a_fortune_on_this_guys_back'] = someone_made_a_fortune_on_this_guys_back(train)

In [ ]:
train